In [13]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns

# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K

# Train-Test
from sklearn.model_selection import train_test_split

# Scaling data
from sklearn.preprocessing import StandardScaler

# Classification Report
from sklearn.metrics import classification_report

from keras.utils import to_categorical

In [126]:
data = pd.read_csv('./final/postop_binary_norm.csv')
X, y = data.iloc[:, :-1], data.iloc[:, -1]
print(X.head(), y.head())

   Age at Operation  Gender  Days Abdominal Pain  Temperature  Tenderness  \
0          0.823529       1             0.047619     0.446809           0   
1          0.764706       0             0.095238     0.468085           0   
2          0.882353       0             0.000000     0.446809           1   
3          0.764706       0             0.047619     0.191489           1   
4          0.882353       1             0.333333     0.659574           0   

   Leukos [10^9/l]  CRP [mg/l]  No of Sonographies  Diameter Appendix [mm]  
0         0.284967    0.015456                0.00                0.263158  
1         0.640042    0.079341                0.00                0.368421  
2         0.353680    0.004894                0.00                0.263158  
3         0.175096    0.000000                0.25                0.052632  
4         0.414719    0.037094                0.50                0.263158   0    0
1    1
2    0
3    0
4    0
Name: Postoperative Diagnosis, dtype: in

In [3]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X = pd.DataFrame(sc.fit_transform(X))
# print(X.head())

          0         1         2         3         4         5         6   
0  1.011211  1.256009 -0.348107 -0.134079  0.888302 -0.630823 -0.638791  \
1  0.741670 -0.796173  0.119830 -0.004641  0.888302  1.320311 -0.201598   
2  1.280752 -0.796173 -0.816043 -0.134079 -0.942209 -0.253246 -0.711069   
3  0.741670 -0.796173 -0.348107 -1.687338 -0.942209 -1.234563 -0.744563   
4  1.280752  1.256009  2.459514  1.160304  0.888302  0.082165 -0.490709   

          7  
0 -0.497849  
1 -0.497849  
2 -0.497849  
3  1.344901  
4  3.187651  


In [131]:
y_cat = to_categorical(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y_cat, test_size=0.2, random_state=42)

In [139]:
model = Sequential()
model.add(Dense(512, input_shape = (9,), activation = "relu"))
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(2, activation = "softmax"))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_99 (Dense)            (None, 512)               5120      
                                                                 
 dense_100 (Dense)           (None, 512)               262656    
                                                                 
 dropout_53 (Dropout)        (None, 512)               0         
                                                                 
 dense_101 (Dense)           (None, 256)               131328    
                                                                 
 dropout_54 (Dropout)        (None, 256)               0         
                                                                 
 dense_102 (Dense)           (None, 128)               32896     
                                                                 
 dropout_55 (Dropout)        (None, 128)             

In [140]:
import tensorflow as tf

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

model.compile(Adam(learning_rate = 0.001), loss="binary_crossentropy", metrics='accuracy')
# model.compile(tf.keras.optimizers.SGD(), loss='categorical_crossentropy', metrics='accuracy')
model.fit(X_train, y_train, verbose=1, epochs=300, callbacks=[callback])


Epoch 1/300
18/18 [==============================] - 1s 4ms/step - loss: 0.5819 - accuracy: 0.7554
Epoch 2/300
18/18 [==============================] - 0s 4ms/step - loss: 0.4921 - accuracy: 0.7860
Epoch 3/300
18/18 [==============================] - 0s 4ms/step - loss: 0.4387 - accuracy: 0.8040
Epoch 4/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3870 - accuracy: 0.8615
Epoch 5/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3803 - accuracy: 0.8507
Epoch 6/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3410 - accuracy: 0.8759
Epoch 7/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3281 - accuracy: 0.8831
Epoch 8/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3196 - accuracy: 0.8867
Epoch 9/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3225 - accuracy: 0.8795
Epoch 10/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3088 - accuracy: 0.8777
Epoch 11/

In [141]:
y_prob = model.predict(X_test)
y_pred_class = y_prob.argmax(axis=-1)

from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix(y_test_class, y_pred_class)

5/5 [==============================] - 0s 1ms/step


array([[107,  14],
       [  9,   9]], dtype=int64)

In [142]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90       121
           1       0.39      0.50      0.44        18

    accuracy                           0.83       139
   macro avg       0.66      0.69      0.67       139
weighted avg       0.85      0.83      0.84       139



In [117]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.93      0.88      0.91       125
           1       0.59      0.65      0.62        26
           2       0.10      0.17      0.12         6

    accuracy                           0.82       157
   macro avg       0.54      0.57      0.55       157
weighted avg       0.84      0.82      0.83       157

